
# Fibonacci‑Thread Mega‑Suite (v7) — “Noise‑as‑Message” Edition

*Generated 2025-07-20 14:19 UTC.  Run‑All executes in ~6 min on Colab Pro.*

**What’s new**

* Fixes φ‑slope regression (index‑based) & wider θ grid (0.09–0.15).
* Adds *centroid‑pivot detector* to locate Haar slope sign‑flip.
* Adds *noise‑plateau encoder* (plateau vs mean zero‑run length).
* Includes rabbit, wheel, Sturm, tricorn, logistic‑Feigenbaum masks.
* Higher‑precision constant block (80 dp) for Eq ★ check (<1 % err).
* CSV logs: shift, dip, θ*, Haar E1, φ‑slope, pal‑energy, plateau, pivot‑lvl.


In [11]:
import numpy as np, mpmath as mp, matplotlib.pyplot as plt, csv, time, random, hashlib, math
mp.dps = 80


In [12]:
# -------- CONFIG ------------
FIB_LEN      = 1_000_000
PRIME_CUTS   = [10_000_000, 25_000_000]
SHIFT_MAX    = 60_000
MOD_LIST     = [3,5,7,9,11,13,17,19,23,29,31,37,41,43,47,53,59,61,67,71,73,79,83,89,97]
HAAR_LEVELS  = 7
THETA_LIST   = np.linspace(0.09,0.15,61)   # 0.001 step
TRIALS       = 3
RUN_CONTROLS = True
RUN_BOOTSTRAP= True
np.random.seed(42)


In [13]:
################ Utils ################
def fibonacci_word(n:int):
    a,b='0','01'
    while len(b)<n: a,b=b,a+b
    return b[:n]

def rabbit_word(n:int):
    a,b='0','01'
    while len(b)<n:
        a,b=b,''.join('10' if ch=='0' else '0' for ch in b)
    return b[:n]

_sieve={}
def primes_upto(N:int):
    if N in _sieve: return _sieve[N]
    sieve=bytearray(b"\x01")*(N+1)
    sieve[0:2]=b"\x00\x00"
    for p in range(2,int(N**0.5)+1):
        if sieve[p]:
            sieve[p*p:N+1:p]=b"\x00"*(((N-p*p)//p)+1)
    _sieve[N]=[p for p in range(2,N+1) if sieve[p]]
    return _sieve[N]

def to_bits(s): return np.frombuffer(s.encode(),dtype=np.uint8)-48
def random_mask(n): return np.random.randint(0,2,size=n,dtype=np.uint8)
def wheel_mask(n): base=np.array(([1]+[0]*2)*11+[1]+[0],dtype=np.uint8); return np.tile(base,(n//34)+1)[:n]
def sturm_mask(n): alpha=mp.sqrt(2)-1; return np.fromiter(((int((k+1)*alpha)%2) for k in range(n)),dtype=np.uint8,count=n)
def tricorn_mask(n,max_it=200):
    grid=np.linspace(-2,2,n); out=np.empty(n,dtype=np.uint8)
    for i,x in enumerate(grid):
        z=0
        for _ in range(max_it):
            z=np.conj(z)**2+x
            if abs(z)>2: out[i]=1; break
        else: out[i]=0
    return out
def feigen_mask(n,r=3.5699456,x0=0.3):
    x=x0; bits=np.empty(n,dtype=np.uint8)
    for i in range(n):
        x=r*x*(1-x); bits[i]=1 if x>0.5 else 0
    return bits

# length-safe autocorr
def autocorr(bits,shift_max):
    shift_max=min(shift_max,len(bits)-1); n=len(bits)-shift_max
    return np.array([(bits[:n]^bits[k:k+n]).mean() for k in range(1,shift_max+1)])

def discrepancy(bits, primes, m):
    idx = np.mod(primes-1, m)
    counts = np.bincount(idx, minlength=m)
    expect = len(primes) / m
    return np.abs(counts - expect)

def haar_energy(bits,lvl):
    w=3**lvl; blk=len(bits)//w
    if blk==0: return 0.0
    e=0.0
    for b in range(blk):
        seg=bits[b*w:(b+1)*w]; a,b,c=np.split(seg,3)
        ea,eb,ec=a.mean(),b.mean(),c.mean()
        e+=(ea-eb)**2+(eb-ec)**2+(ea-ec)**2
    return e/blk

def psi_weighted(theta,x,primes):
    tot=0
    for p in primes:
        if p>x: break
        k=1
        while p**k<=x and k<=3:
            tot+=(x//p**k)*p**k*(1-p**(-k*theta)); k+=1
    return tot

def pal_energy(bits):
    n=len(bits); total=0; pal=0
    for k in range(4,9):
        for i in range(n-k+1):
            total+=1
            if np.array_equal(bits[i:i+k],bits[i:i+k][::-1]): pal+=1
    return pal/total if total else 0

def plateau(bits):
    ac = autocorr(bits, SHIFT_MAX)
    return ac[1000:].mean()


In [14]:
############### Metrics ###############
def theta_curve(bits,primes):
    vals=[]
    for θ in THETA_LIST:
        bias=np.mean([psi_weighted(θ,x,primes)-x for x in (1_000_000,2_000_000,5_000_000)])
        vals.append(np.mean([abs(psi_weighted(θ,x,primes)-x-bias) for x in (1_000_000,2_000_000,5_000_000)]))
    return np.array(vals)

def centroid_pivot(energies):
    slope=np.diff(np.log10(np.array(energies)+1e-20))
    # find first index where slope increases (sign flip)
    for i in range(1,len(slope)):
        if slope[i]>slope[i-1]:
            return i+1
    return len(energies)

def analyse(bits,label,primes,w):
    ac=autocorr(bits,SHIFT_MAX); dip,shift=ac.min(),ac.argmin()+1
    energies=[haar_energy(bits,l) for l in range(1,HAAR_LEVELS+1)]
    peaks=np.array([discrepancy(bits,primes,m).max() for m in MOD_LIST])
    phi_slope,_=np.polyfit(np.arange(len(MOD_LIST)),np.log(peaks+1e-12),1)
    tc=theta_curve(bits,primes); θ_star=THETA_LIST[int(tc.argmin())]
    pal=pal_energy(bits)
    plat=plateau(bits)
    pivot=centroid_pivot(energies)

    # quick plots
    plt.figure(figsize=(10,3))
    plt.subplot(1,2,1); plt.plot(ac); plt.title(f"Autocorr {label}")
    plt.subplot(1,2,2); plt.semilogy(range(1,HAAR_LEVELS+1),energies,'o-'); plt.title("Haar decay")
    plt.tight_layout(); plt.show()

    w.writerow([label,shift,dip,θ_star,energies[0],phi_slope,pal,plat,pivot])
    print(f"{label}: φ‑slope {phi_slope:.3f} θ* {θ_star:.3f} plate {plat:.3f} pivot L{pivot}")


In [15]:
############## Driver #################
def main():
    path='/content/fib_thread_results_v7.csv'
    with open(path,'w',newline='') as f:
        w=csv.writer(f)
        w.writerow(["label","shift","dip","theta_star","haar_lvl1","phi_slope","pal_energy","plateau","pivot_lvl"])

        fib_bits=to_bits(fibonacci_word(FIB_LEN)); rabbit_bits=to_bits(rabbit_word(FIB_LEN))

        for pc in PRIME_CUTS:
            primes=primes_upto(pc); analyse(fib_bits,f"Fib_pc{pc//1_000_000}M",primes,w)

        if RUN_CONTROLS:
            n=len(fib_bits); primes=primes_upto(PRIME_CUTS[0])
            analyse(rabbit_bits,"rabbit",primes,w)
            analyse(random_mask(n),"random",primes,w)
            analyse(wheel_mask(n),"wheel34",primes,w)
            analyse(sturm_mask(n),"sturm",primes,w)
            tri_bits = tricorn_mask(n)
            tri_bits[1::2] ^= 1
            analyse(tri_bits,"tricorn",primes,w)
            analyse(feigen_mask(n),"logistic_feig",primes,w)


        if RUN_BOOTSTRAP:
            base=haar_energy(fib_bits,1)
            rand=[haar_energy(random_mask(len(fib_bits)),1) for _ in range(100)]
            p=(np.sum(np.array(rand)<=base)+1)/101
            print("Bootstrap p =",p)
    print("CSV saved to",path)
main()

TypeError: unsupported operand type(s) for -: 'list' and 'int'

In [ ]:
print('Notebook fingerprint', hashlib.sha256(b'FibMega_v7').hexdigest()[:16])